# Application of RCNN

## The Task
To predict the next position of the box as shown in the yellow, this box will be moving to and fro in the shown frame.
![box predict](../figures/box_predict.png)

## Why it s difficult?
Based on current positon of the box the box can have two postion, one is in the right of the box and one is in the left of the box. as shown in the below diagram.
The CNN takes in to consideration the current frame without any previous information about the direction in which the box was moving. Where as the RCNN predict the next posrion of the box by taking considering input from many previous frames.
![box predict](../figures/abig_predict.png)



## How to solve it.
CNN combined with the temporal memory can solve this problem, This combination is commonly known as the Recurrent-CNN architecture.
![box memory](../figures/temporal_prediction.png)


### Import libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw

In [ ]:
FRAME_SIZE = [5,50] # size of bar in which the yellow box will move
BOX_WIDTH = 3 # height and width of the yellow box

### Functions to draw box in matrix

In [ ]:
def get_rect(x, y, width, height):
    """
    making rectlangle
    """
    rect = np.array([(0, 0), (width-1, 0), (width-1, height-1), (0, height-1), (0, 0)])
    offset = np.array([x, y])
    transformed_rect = rect + offset
    return transformed_rect

def get_array_with_box_at_pos(x):
    """
    incorporating rectanle in to frame
    """
    data = np.zeros(FRAME_SIZE)
    img = Image.fromarray(data)
    draw = ImageDraw.Draw(img)
    rect = get_rect(x=x, y=1, width=BOX_WIDTH, height=BOX_WIDTH)
    draw.polygon([tuple(p) for p in rect], fill=1)
    new_data = np.asarray(img)
    return new_data

If we want the box to be at the position 20 then the resultant bar with the box will look like as given below.

## Use functions to generate data sequence

In [ ]:
# defining movement range, the box can move from 1 to FRAME_SIZE - BOX_WIDTH
movement_offset = 1
movement_start = movement_offset
movement_end = (FRAME_SIZE[1]-1) - BOX_WIDTH
movement_range = movement_end - movement_offset
movement_start, movement_end, movement_range

### Create movement pattern

In [ ]:
DATA_POINTS = 100 # max number of data to be prepared

Creating sine like function, the postion of data in cycle represent the position of the box in the bar

In [ ]:
base = (np.arange(DATA_POINTS)/DATA_POINTS)* 6 *np.pi # 6 for 6 up and down in the cycle
sined = (np.sin(base) + 1 )/2
plt.scatter(base, sined)
plt.show()

In [ ]:
def sin_to_pos(sin_val):
    return (sin_val*movement_range)+movement_offset

In [ ]:
frames = []
print_every_n_frames = 2
for i,t in enumerate(sined):
    frame = get_array_with_box_at_pos(sin_to_pos(t))
    if(i % print_every_n_frames)==0:
        plt.imshow(frame, interpolation='nearest')
        plt.show()
    frames.append(frame)

## Export the data to file

In [ ]:
y = sin_to_pos(sined[1:])
X = frames[:-1]
print("Lenght of X : ",len(X), " | Length of Y : ",len(y))

In [ ]:
with open('RCNN/data/sliding_square_target.npy', 'wb') as file:
    np.save(arr=y, file=file)

with open('RCNN/data/sliding_square.npy', 'wb') as file:
    np.save(arr=X, file=file)
